Scrape reviews together with rating

In [418]:
#load necessary libraries
from selenium import webdriver
import time
import chromedriver_binary
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
import pandas as pd
import re


#disable build check to if latest version of Chrome is not supported
service = Service(service_args=['--disable-build-check'])

#open the window size to simulate mobile view
options = webdriver.ChromeOptions()
options.add_argument("window-size=600,800")
driver = webdriver.Chrome(service=service, options=options)


The chromedriver version (128.0.6582.0) detected in PATH at C:\Users\22507\anaconda3\lib\site-packages\chromedriver_binary\chromedriver.exe might not be compatible with the detected chrome version (126.0.6478.127); currently, chromedriver 126.0.6478.126 is recommended for chrome 126.*, so it is advised to delete the driver in PATH and retry


In [390]:
# restaurant_names = []
#
# try:
#     names = driver.find_elements(By.CSS_SELECTOR, "h1")
#     for name in names:
#         restaurant_names.append(name.text)
# except:
#     pass
#
# restaurant_names

In [391]:
# stars = []
# try:
#     numbers = driver.find_elements(By.CSS_SELECTOR, 'div.arrange-unit__09f24__rqHTg.y-css-1iy1dwt .y-css-9tnml4[role="img"][aria-label*="star rating"]')
#     for number in numbers:
#         aria_label = number.get_attribute("aria-label")
#         star = aria_label.split()[0]
#         stars.append(star)
# except:
#     pass
#
# stars

In [392]:
# review_contents = []
#
# try:
#     reviews = driver.find_elements(By.CSS_SELECTOR, 'p.comment__09f24__D0cxf.y-css-h9c2fl')
#     for review in reviews:
#         clean_text = review.text.replace('\n', ' ')
#         review_contents.append(clean_text)
# except:
#     pass
#
# review_contents

In [393]:
# review_dates = []
#
# try:
#     review_elements = driver.find_elements(By.CSS_SELECTOR, 'span.y-css-wfbtsu')
#
#     # 定义一个日期的正则表达式模式，如Apr 27, 2024
#     date_pattern = re.compile(r'\b[A-Za-z]{3} \d{1,2}, \d{4}\b')
#
#     for review_element in review_elements:
#         text = review_element.text
#         if date_pattern.match(text):
#             review_dates.append(text)
# except:
#     pass
#
# review_dates

In [394]:
# length = len(stars)
# try:
#     name = driver.find_element(By.CSS_SELECTOR, "h1")
#     restaurant_name = name.text
#     restaurant_names = [restaurant_name] * length
# except:
#     pass
#
# restaurant_names

In [419]:
def get_all_reviews(url):

    driver.get(url)

    page_number = 1

    restaurant_name = ""
    restaurant_names = []
    review_contents = []
    stars = []
    review_dates = []

    # scrape each page
    def scrape_page():

        # Review Date
        try:
            review_elements = driver.find_elements(By.CSS_SELECTOR, 'span.y-css-wfbtsu')
            date_pattern = re.compile(r'\b[A-Za-z]{3} \d{1,2}, \d{4}\b')
            for review_element in review_elements:
                text = review_element.text
                if date_pattern.match(text):
                    review_dates.append(text)
        except:
            pass

        # Review's Rating
        try:
            numbers = driver.find_elements(By.CSS_SELECTOR,
                                           'div.arrange-unit__09f24__rqHTg.y-css-1iy1dwt .y-css-9tnml4[role="img"][aria-label*="star rating"]')
            for number in numbers:
                aria_label = number.get_attribute("aria-label")
                star = aria_label.split()[0]
                stars.append(star)
        except:
            pass

        # Review Contents
        try:
            reviews = driver.find_elements(By.CSS_SELECTOR, 'p.comment__09f24__D0cxf.y-css-h9c2fl')
            for review in reviews:
                clean_text = review.text.replace('\n', ' ')
                review_contents.append(clean_text)
        except:
            pass


    # scrape first page
    time.sleep(5)
    try:
        print('Page 1 ...')
        scrape_page()
    except TimeoutException:
        print("Timeout while waiting for the first page elements to load.")

    # scrape other pages
    while True:
        try:
            # click next page button
            load_next_page_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH,
                                            '//a[@class="next-link navigation-button__09f24__m9qRz y-css-7ln3jw" and @aria-label="Next"]'))
            )

            # able to click
            if load_next_page_button.is_enabled():
                load_next_page_button.click()
                print("Clicked 'Next' button...Loading...")
                page_number += 1
                print(f"Page {page_number} ...")

                time.sleep(3)

                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'span.y-css-wfbtsu'))
                )
                scrape_page()
            else:
                print("Next button is either not displayed or not enabled")
                break

        except TimeoutException:
            print(f"No more 'Next' button found. Stopping at page {page_number}.")
            break

    # Restaurant Names
    length = len(stars)
    try:
        name = driver.find_element(By.CSS_SELECTOR, "h1")
        restaurant_name = name.text
        restaurant_names = [restaurant_name] * length
    except:
        pass

    review_list = zip(restaurant_names, stars, review_dates, review_contents)

    return review_list

In [420]:
holy_crab_reviews = get_all_reviews('https://www.yelp.com/biz/holycrab-singapore?osq=Restaurants')
holy_crab_reviews

Page 1 ...
Clicked 'Next' button...Loading...
Page 2 ...
Clicked 'Next' button...Loading...
Page 3 ...
No more 'Next' button found. Stopping at page 3.


In [421]:
df = pd.DataFrame(holy_crab_reviews, columns=['Restaurant Name', 'Review Rating', 'Review Date', 'Review Content'])
df

,Restaurant Name,Review Rating,Review Date,Review Content
0,Holycrab,4,"Apr 27, 2024",You must try chili crab in Singapore if you're...
1,Holycrab,5,"Jan 21, 2024",I had to get Chili Crab while I was in Singapo...
2,Holycrab,5,"Apr 13, 2023",My gf found this place on yelp and saw the ama...
3,Holycrab,1,"Feb 5, 2024","We went there on February the 5th, at about 1p..."
4,Holycrab,5,"Dec 6, 2023",One of the best places to try chili crab. They...
5,Holycrab,5,"Oct 17, 2022",I just had the best crab in my life at Holy Cr...
6,Holycrab,3,"Dec 26, 2022",Not quite my cup of tea as seafood was never m...
7,Holycrab,3,"Dec 16, 2023","Food is average, chili crab is so-so. Service ..."
8,Holycrab,5,"Sep 3, 2022",I suggest making reservations. I called last m...
9,Holycrab,5,"Nov 4, 2019",If your desire for Singapore's supposed classi...


In [399]:
driver.quit()